<a href="https://colab.research.google.com/github/ItayShalit/Intro-to-Deep-Learning/blob/main/Intro_to_DL_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1


### Data Loading

In [21]:
import torch
import torchvision
import torchvision.transforms as transforms
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np


# transform = transforms.Compose(
#     [transforms.ToTensor(),
#      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

TRAIN_SIZE = 5000
TEST_SIZE = 1000
BATCH_SIZE = 4

# trainSampler = torch.utils.data.RandomSampler(TRAIN_SIZE)
# testSampler = torch.utils.data.RandomSampler(TEST_SIZE)



trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, 
                                        transform = transforms.ToTensor())
sample_train_idx = torch.randint(len(trainset), size=(TRAIN_SIZE,))
#trainSampler = torch.utils.data.SubsetRandomSampler(indices = sample_train_idx)
trainSampler = torch.utils.data.WeightedRandomSampler(weights = [1 for i in range(TRAIN_SIZE)], replacement = True, num_samples = TRAIN_SIZE)

# trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size,
#                                           num_workers=2, shuffle = False, sampler = trainSampler)

testset = torchvision.datasets.CIFAR10(root = './data', train = False,
                                       download = True, transform = transforms.ToTensor())
sample_test_idx = torch.randint(len(trainset), size=(TEST_SIZE,))
# testSampler = torch.utils.data.SubsetRandomSampler(indices = sample_test_idx)
# testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size,
#                                          num_workers = 2, shuffle = True)

# classes = ('plane', 'car', 'bird', 'cat',
#            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


### Training an SVM Classifier 

In [37]:
MAX_INTENSITY = 255

train_x = trainset.data/MAX_INTENSITY
train_y = trainset.targets

test_x = testset.data/MAX_INTENSITY
test_y = testset.targets

train_x = train_x.reshape(train_x.shape[0], -1)
train_x = train_x[sample_train_idx,:]
train_y = np.array(train_y)[sample_train_idx]

test_x = test_x.reshape(test_x.shape[0], -1)
test_x = test_x[sample_test_idx,:]
test_y = np.array(test_y)[sample_test_idx]

#### Hyperparameter Tuning

In [ ]:
param_grid1 = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf']}
param_grid2 = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['linear']}

grid1 = GridSearchCV(SVC(),param_grid1, refit=True, verbose=2, n_jobs = -1)
grid2 = GridSearchCV(SVC(),param_grid1, refit=True, verbose=2, n_jobs = -1)

grid1.fit(train_x, train_y)
grid2.fit(train_x, train_y)

#### Training the Models

In [ ]:
params1 = grid1.best_params_
params2 = grid2.best_params_

rbf_model = SVC(C = params1["C"], kernel = "rbf", gamma = params1["gamma"])
linear_model = SVC(C = params2["C"], kernel = "linear", gamma = params2["gamma"])

rbf_model.fit(train_x, train_y)
linear_model.fit(train_x, train_y)

test_pred_rbf = rbf_model.predict(test_x)
test_pred_linear = linear_model.predict(test_x)

print(f"""accuracy score for linear model is: {accuracy_score(test_y, test_pred_linear)} \n 
        accuracy score for rbf model is: {accuracy_score(test_y, test_pred_rbf)}""")
